In [1]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context 

In [2]:
from urllib.request import urlopen # Website connections (open the website)
from bs4 import BeautifulSoup # For HTML parsing
from lxml import html # To process HTML language in Python
import unicodecsv as csv # In order to create a csv file to store the information
import pandas as pd # For converting results to a dataframe
from tqdm import tqdm # Show progress of the loop
from lxml import html # For parsing html
from tqdm import tqdm # Progress bar for loops
import time # Measures time
from datetime import datetime # Measures time

In [3]:
def get_soup (url):
    '''
    Accesses and reads the url then returns the html of the url
    '''
    html = urlopen(url).read()
    soup = BeautifulSoup(html,'lxml')
    return soup

In [4]:
def find_between(s, first, last):
    '''
    Finds all the strings between the second and third parameters
    '''
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except:
        return ""

In [5]:
def change_date_format(txt):   
    '''
    Changes all the string format from a to b (a,b)
    '''
    txt = txt.replace('Jan', 'January')
    txt = txt.replace('Feb', 'February')
    txt = txt.replace('Mar', 'March')
    txt = txt.replace('Apr', 'April')
    txt = txt.replace('Jun', 'June')
    txt = txt.replace('Jul', 'July')
    txt = txt.replace('Aug', 'August')
    txt = txt.replace('Sep', 'September')
    txt = txt.replace('Oct', 'October')
    txt = txt.replace('Nov', 'November')
    txt = txt.replace('Dec', 'December')
    return txt

In [65]:
def get_campground_info(url):
    '''
    Accesses one campground url and then gets all the info needed from the url and saves them
    All of the variable names represent what information was grabbed from the url
    '''
    camp = get_soup(url)
    new_div = camp.new_tag('p')
    new_div.string = ", "
    
    address_text = camp.find_all('div', {"class":"eight columns bg-white p20"})[0].find_all('div',{'class':'float-left'})[0]
    address_text.insert(4, new_div)
    address_text_edited = address_text.text
    address = find_between(address_text_edited, '\n\r\n                        ', '\r')
    
    geocode_text = camp.find_all('div', {"class":"eight columns bg-white p20"})[0].find_all('div',{'class':'float-left'})[1].text
    geocode = find_between(geocode_text, 'GPS Information: ', '\r\n').split(', ')
    
    open_date = change_date_format(camp.find('div', {"class":"four columns bg-white p20"}).find('p').text)
    min_date = ''
    max_date = ''
    if '.' or '!' in open_date:
        if 'Currently closed' in open_date:
            min_date = datetime.strptime(find_between(open_date, 'Open ', ' -') + ", 2020", "%B %d, %Y")
            max_date = datetime.strptime(find_between(open_date, '- ', open_date[-1]) + open_date[-1] + ", 2020", "%B %d, %Y")
        else:
            min_date = datetime.strptime(find_between(open_date, '! ', ' -') + ", 2020", "%B %d, %Y")
            max_date = datetime.strptime(find_between(open_date, '- ', open_date[-1]) + open_date[-1] + ", 2020", "%B %d, %Y")
    camp_open_date = [min_date, max_date]
    
    full_season = camp.find('div', {"class":"four columns bg-white p20"}).find('table').find_all('tr')[0].text
       
    if 'No' in full_season:
        full_season = 0
    else:
        full_season = 1
        
    overnight_site = camp.find('div', {"class":"four columns bg-white p20"}).find('table').find_all('tr')[1].text
    
    if 'No' in overnight_site:
        overnight_site = 0
    else:
        overnight_site = 1
        
    roofed_lodging = camp.find('div', {"class":"four columns bg-white p20"}).find('table').find_all('tr')[2].text
    
    if 'No' in roofed_lodging:
        roofed_lodging = 0
    else:
        roofed_lodging = 1
        
    tenting_available = camp.find('div', {"class":"four columns bg-white p20"}).find('table').find_all('tr')[3].text
    
    if 'No' in tenting_available:
        tenting_available = 0
    else:
        tenting_available = 1
    
    site_type = [full_season, overnight_site, roofed_lodging, tenting_available]
    total_sites = camp.find_all('div',{'class':'numbers'})[0].text.replace(' ','').replace('\r\n','')
    overnight_sites = camp.find_all('div',{'class':'numbers'})[1].text.replace(' ','').replace('\r\n','')
    seasonal_sites = camp.find_all('div',{'class':'numbers'})[2].text.replace(' ','').replace('\r\n','')
    max_amperage = camp.find_all('div',{'class':'numbers'})[3].text.replace(' ','').replace('\r\n','')
    electrical_water_sewer = camp.find_all('div',{'class':'numbers'})[4].text.replace(' ','').replace('\r\n','')
    electrical_water = camp.find_all('div',{'class':'numbers'})[5].text.replace(' ','').replace('\r\n','')
    electrical = camp.find_all('div',{'class':'numbers'})[6].text.replace(' ','').replace('\r\n','')
    no_services = camp.find_all('div',{'class':'numbers'})[7].text.replace(' ','').replace('\r\n','')
    max_length_of_rv = camp.find_all('div',{'class':'numbers'})[9].text.replace(' ','').replace('\r\n','').replace('\xa0ft', ' ').replace(' ', '')
    
    campground_details = [total_sites, overnight_sites, seasonal_sites, max_amperage, electrical_water_sewer, electrical_water, electrical, no_services, max_length_of_rv]
    
    activities = camp.find_all('div', {'class':'mb15'})[5].find('p').text
    
    facilities = []
    for i in range(24):
        facility_name = camp.find_all('div', {'class':'mb15'})[-1].find_all('span')[i].text
        if not camp.find_all('div', {'class':'mb15'})[-1].find_all('span')[i].has_attr('disabled'):
            facility_available = 1
        else:
            facility_available = 0
        facilities.append(facility_available)
            

    return address, geocode, camp_open_date, site_type, campground_details, activities, facilities

In [96]:
def get_all_campgrounds(url):
    '''
    Finds all the campgrounds from the selected url and then grabs all the campgrounds
    Then with collaboration with the function above, it goes into each url and runs the above function
    Then loops around each campgrounds then saves all the scraped info to a dictionary which will be outputted 
    '''
    
    campgrounds = get_soup(url)
    # Finding all campgrounds
    campgrounds_num = len(campgrounds.find_all('div', {"class":"twelve columns campg"}))
    all_campground_dictionary = []
    failed_lists = []
    all_activity_lists=[]

    # Accesssing each campground and grabbing the info using function above, then saves them to a dictionary
    for i in tqdm(range(campgrounds_num)):
        try:
            camp_name = campgrounds.find_all('div', {"class":"twelve columns campg"})[i].find('h2').text
            campground_url_extra = campgrounds.find_all('div', {"class":"twelve columns campg"})[i].find('div', {"class":"camp-link wrapbox"})['onclick']
            camp_url = 'https://campinginontario.ca' + find_between(campground_url_extra, "'", "'")

            camp = get_campground_info(camp_url)
            address = camp[0]
            postal_code = address.split(', ')[-1]
            lat = camp[1][0]
            long = camp[1][1]
            
            min_date = camp[2][0]
            max_date = camp[2][1]
            
            full_season = camp[3][0]
            overnight_site = camp[3][1]
            roofed_lodging = camp[3][2]
            tenting_available = camp[3][3]
            
            total_sites = camp[4][0]
            overnight_sites = camp[4][1]
            seasonal_sites = camp[4][2]
            max_amperage = camp[4][3]
            electrical_water_sewer = camp[4][4]
            electrical_water = camp[4][5]
            electrical = camp[4][6]
            no_services = camp[4][7]
            max_length_of_rv = camp[4][8]
            
            activities = camp[5]
            
            activity_lists = []
            activity_lists = camp[5].split(', ')
            for activity in activity_lists:
                if activity not in all_activity_lists:
                    all_activity_lists.append(activity)
            
            accessible_facilities = camp[6][0]
            boating = camp[6][1]
            cable_tv = camp[6][2]
            controlled_access = camp[6][3]
            credit_debit_cards = camp[6][4]
            dumping = camp[6][5]
            group_camping = camp[6][6]
            hot_tub = camp[6][7]
            hotspot = camp[6][8]
            wifi_at_site = camp[6][9]
            laundromat = camp[6][10]
            pet_friendly = camp[6][11]
            planned_activities = camp[6][12]
            playground = camp[6][13]
            propane = camp[6][14]
            pull_thru_sites = camp[6][15]
            rec_hall = camp[6][16]
            restaurant = camp[6][17]
            store = camp[6][18]
            indoor_pool = camp[6][19]
            lake_river_beach = camp[6][20]
            outdoor_pool = camp[6][21]
            toilets_pit_outside = camp[6][22]
            toilets_showers = camp[6][23]
            
            # Dictionary to save all the info
            camp_img_url = 'https://campinginontario.ca' + campgrounds.find_all('div', {"class":"twelve columns campg"})[i].find('img')['src']
            d = {
                'camp_name': camp_name,
                'camp_url': camp_url,
                'camp_img_url': camp_img_url,
                'address': address,
                'postal_code': postal_code,
                'lat': lat,
                'long': long,
                'earliest_open_date': min_date,
                'latest_open_date': max_date,
                'full_season': full_season,
                'overnight_site': overnight_site,
                'roofed_lodging': roofed_lodging,
                'tenting_available': tenting_available,
                'total_sites':total_sites,
                'overnight_sites':overnight_sites,
                'seasonal_sites':seasonal_sites,
                'max_amperage':max_amperage,
                'electrical_water_sewer':electrical_water_sewer,
                'electrical_water':electrical_water,
                'electrical':electrical,
                'no_services':no_services,
                'max_length_of_rv':max_length_of_rv,
                'activities':activities,
                'accessible_facilities':accessible_facilities,
                'boating':boating,  
                'cable_tv':cable_tv, 
                'controlled_access':controlled_access, 
                'credit_debit_cards':credit_debit_cards, 
                'dumping':dumping, 
                'group_camping':group_camping,
                'hot_tub':hot_tub, 
                'hotspot':hotspot, 
                'wifi_at_site':wifi_at_site, 
                'laundromat':laundromat, 
                'pet_friendly':pet_friendly, 
                'planned_activities':planned_activities, 
                'playground':playground, 
                'propane':propane,
                'pull_thru_sites':pull_thru_sites, 
                "rec_hall":rec_hall, 
                "restaurant":restaurant, 
                "store":store,
                "indoor_pool":indoor_pool,
                "lake_river_beach":lake_river_beach,
                "outdoor_pool":outdoor_pool, 
                "toilets_pit_outside":toilets_pit_outside,
                "toilets_showers":toilets_showers
                }
            all_campground_dictionary.append(d)
            time.sleep(0.5)
        except:
            camp_name = campgrounds.find_all('div', {"class":"twelve columns campg"})[i].find('h2').text
            campground_url_extra = campgrounds.find_all('div', {"class":"twelve columns campg"})[i].find('div', {"class":"camp-link wrapbox"})['onclick']
            camp_url = 'https://campinginontario.ca' + find_between(campground_url_extra, "'", "'")
            failed_lists.append(camp_url)            
            pass
    return all_campground_dictionary, failed_lists, all_activity_lists

In [97]:
# Running the function to get all info then saves to a variable
run_get_all_camps = get_all_campgrounds('https://www.campinginontario.ca/campgrounds?region=1,2,18,5,8,9,10,11,12,13,14,15,16,17')

100%|██████████| 412/412 [10:59<00:00,  1.60s/it]


In [95]:
# Gets all the activities separately for filtering within UI
all_activities = run_get_all_camps[2]
print(all_activities)
print(f'Number of all activities: {len(all_activities)}')
df = pd.DataFrame(all_activities)
df.columns = ['activities']
df = df[df.activities != '']
df.to_csv('activity_lists.csv', index=False)
df

['Casino', "Farmers' market/stands", 'Golf (mini)', 'Movies', 'Museum/heritage sites', 'Pedal carts', 'Restaurants', 'Shopping', 'Boat tours (scenic)', 'Fishing', 'Golf', 'Scuba diving', 'Walking/hiking trails', 'Ziplining', 'Table tennis', 'Volleyball', 'Basketball', 'Boat rental (non-motorized)', 'Swimming (lake', 'river)', 'ATV-friendly', 'Horseshoes', 'Motorboats allowed', 'Shuffleboard', 'Cycling', 'Water skiing', 'Wildlife viewing/birding', 'Spa', 'Nude beach', 'Day use', 'Mountain biking', 'Wine tours', 'Boat rental (motorized)', 'Badminton', 'Theatre (live)', 'Water park', 'Dances', 'Horseback riding', 'Baseball', 'Amusement park', 'Rock climbing/bouldering', 'Soccer', '', 'Snowmobiling', 'Chess', 'Fitness', 'Vehicle racing', 'Tennis', 'Fishing (guided)', 'Spelunking/cave exploration', 'Skateboard/rollerblade park', 'Music Festival', 'Horse racing', 'Hunting', 'Go-karts', 'Whitewater rafting']
Number of all activities: 56


,activities
0,Casino
1,Farmers' market/stands
2,Golf (mini)
3,Movies
4,Museum/heritage sites
5,Pedal carts
6,Restaurants
7,Shopping
8,Boat tours (scenic)
9,Fishing


In [77]:
# Failed URLs due to Server Errors except for the first one, which also had a server error when the script was last run but now is working
failed_urls = run_get_all_camps[1]
print(failed_urls)
print(f'Number of all failed urls {len(failed_urls)}')

['https://campinginontario.ca/HappyGreenAcres', 'https://campinginontario.ca/LovesickLakeBeachResort', 'https://campinginontario.ca/BrightonKOAHoliday', 'https://campinginontario.ca/IdyllGlenRVPark', 'https://campinginontario.ca/MapleGroveResortInc', 'https://campinginontario.ca/PineRidgeParkCampgroundRusticCabinRental']
Number of all failed urls 6


In [78]:
# Saves all the results to csv
data = run_get_all_camps[0]
df = pd.DataFrame(data)
df.to_csv('campground_lists.csv', index=False)

In [79]:
df # Example of how the file would look like

,camp_name,camp_url,camp_img_url,address,postal_code,lat,long,earliest_open_date,latest_open_date,full_season,...,propane,pull_thru_sites,rec_hall,restaurant,store,indoor_pool,lake_river_beach,outdoor_pool,toilets_pit_outside,toilets_showers
0,1000 Islands/Kingston KOA,https://campinginontario.ca/1000IslandsKingsto...,https://campinginontario.ca/Uploads/Campground...,"2039 Cordukes Rd, Kingston, Ontario, K7L 4V4",K7L 4V4,44.30151,-76.57545,2020-05-01,2020-10-15,1,...,0,1,1,1,1,0,0,1,0,1
1,1000 Islands/Mallorytown KOA Holiday Campground,https://campinginontario.ca/1000IslandsMallory...,https://campinginontario.ca/Uploads/Campground...,"1477 County Rd 2, PO Box 29, Mallorytown, Onta...",K0E 1R0,44.46956,-75.87978,2020-04-10,2020-11-01,1,...,1,1,1,0,1,0,0,1,0,1
2,Almaguin Campground,https://campinginontario.ca/AlmaguinCampground,https://campinginontario.ca/Uploads/Campground...,"419 Owl Lake Rd, Katrine, Ontario, P0A 1L0",P0A 1L0,45.57136,-79.32835,2020-05-15,2020-10-15,1,...,0,1,1,0,1,0,1,1,0,1
3,Alpine RV Resort,https://campinginontario.ca/AlpineRVResort,https://campinginontario.ca/Uploads/Campground...,"46 Alpine St, Lindsay, Ontario, K9V 4R1",K9V 4R1,44.39709,-78.75377,2020-05-15,2020-10-12,1,...,0,1,1,0,1,0,0,1,0,1
4,Amazing Rocky Park Campground,https://campinginontario.ca/AmazingRockyParkCa...,https://campinginontario.ca/Uploads/Campground...,"423059 Rocky Saugeen Rd, RR 1, Durham, Ontario...",N0G 1R0,44.23216,-80.8259,2020-04-10,2020-10-18,1,...,0,1,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,Woodcrest Resort Park,https://campinginontario.ca/WoodcrestResortPark,https://campinginontario.ca/images/camping/ful...,"1570 Myers Cave Rd, RR 1, Harlowe, Ontario, K0...",K0H 1B0,44.79585,-77.09347,2020-05-01,2020-10-31,1,...,0,1,1,1,1,0,1,0,1,1
402,Woodland Camp Site,https://campinginontario.ca/WoodlandCampSite,https://campinginontario.ca/images/camping/ful...,"209 Allen's Rd, Lakehurst, Ontario, K0L 1J0",K0L 1J0,44.49966,-78.4315,2020-05-07,2020-10-15,1,...,0,1,0,0,0,0,1,0,1,1
403,Woodland Park – Arden,https://campinginontario.ca/WoodlandParkArden,https://campinginontario.ca/images/camping/ful...,"1111 Woodland Park Drive, Central Frontenac, A...",K0H 1B0,44.68716,-76.98459,2020-05-15,2020-10-12,1,...,1,0,0,0,0,0,1,0,1,1
404,Woodland RV Park,https://campinginontario.ca/WoodlandRVPark,https://campinginontario.ca/images/camping/ful...,"266 Townline St 16, St Williams, Ontario, N0E 1P0",N0E 1P0,42.67199,-80.41872,2020-05-01,2020-10-31,1,...,0,1,1,0,1,0,0,1,0,1
